In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-10-21 06:48:12--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  66.3MB/s    in 0.4s    

2022-10-21 06:48:12 (66.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
#copy the users dataframe
df1=df_ratings.copy()

#check for users with less than 200 ratings
df2 = df1.groupby(["user"])['rating'].count().to_frame()
df2 = df2.loc[df2.rating>=200]

#get rid of users with less than 200 ratings
df3 = pd.merge(df1,df2,on='user')

#check for books with less than 100 ratings
df4 = df1.groupby(['isbn'])['rating'].count().to_frame()
df5 = df4.loc[df4.rating>100]

#get rid of books with less than 100 ratings
df6 = pd.merge(df3,df5,on='isbn')

#delete useless columns and then get the final pairs of isbns and ratings
del df6["rating_y"]
del df6["rating"]
df = pd.merge(df_books,df6,on="isbn")

#get the final dataframe
df = df.groupby(['isbn','title',"user"],as_index=False)['rating_x'].mean()


In [5]:
df.head()

,isbn,title,user,rating_x
0,002542730X,Politically Correct Bedtime Stories: Modern Ta...,3363,0.0
1,002542730X,Politically Correct Bedtime Stories: Modern Ta...,11676,6.0
2,002542730X,Politically Correct Bedtime Stories: Modern Ta...,12538,10.0
3,002542730X,Politically Correct Bedtime Stories: Modern Ta...,13552,0.0
4,002542730X,Politically Correct Bedtime Stories: Modern Ta...,16795,0.0


In [6]:
#pivot the table to get a vector of distances (ratings) per title
df = df.pivot_table(index="title",columns="user",values="rating_x").fillna(0)
mat = csr_matrix(df.values)
print(mat.shape[1])

888


In [7]:
df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  model = NearestNeighbors(metric = 'cosine', n_neighbors=5, algorithm='auto')
  model.fit(mat)

  for index in range(len(df)):
    if df.index[index]==book:
      break

  recommended_books=[df.index[index],[]]

  distances,indices = model.kneighbors(df.iloc[index,:].values.reshape(1,-1))
  for i in range(1, len(distances.flatten())):
        recommended_books[1].insert(0, [df.index[indices.flatten()[i]], distances.flatten()[i]])
  return recommended_books

get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075],
  ['The Lovely Bones: A Novel', 0.7234864]]]

In [15]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
